In [52]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [77]:
#Testing with 2010 Bronx data
f = open('AllYears/2010_bronx.csv','r')
df = pd.read_csv(f, parse_dates=True)
f.close()

In [84]:
#makesure Sale Date is Date Type
df['SALE DATE'] = pd.to_datetime(df['SALE DATE'])

In [85]:
df2 = df[df['SALE PRICE'] > 1000]

In [86]:
#columns to keep
to_keep =(
[u'BOROUGH', u'NEIGHBORHOOD', u'BUILDING CLASS CATEGORY',
 u'TAX CLASS AT PRESENT', u'BLOCK', u'LOT',
       u'BUILDING CLASS AT PRESENT', u'ADDRESS',
       u'ZIP CODE', u'RESIDENTIAL UNITS', u'COMMERCIAL UNITS', u'TOTAL UNITS',
       u'LAND SQUARE FEET', u'GROSS SQUARE FEET', u'YEAR BUILT',
       u'TAX CLASS AT TIME OF SALE', u'BUILDING CLASS AT TIME OF SALE',
       u'SALE PRICE', u'SALE DATE'])

In [91]:
cleaned = df2[to_keep].reset_index(drop=True)

In [114]:
#create year column
cleaned['YEAR'] = cleaned['SALE DATE'].apply(lambda r: r.year)


cleaned.columns = [i.strip('\n') for i in cleaned.columns]

In [101]:
#Get All Years into Single DataFrame
import glob
import os
path = os.getcwd() + '/AllYears/'
allFiles = glob.glob(path + "/*.csv")


In [115]:
frame = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0)
    
    
    #some have tabs at end of file names
    df.columns = [i.strip('\n') for i in df.columns]
    
    
    
    
    
    list_.append(df)
frame = pd.concat(list_,)

In [116]:
frame.count()

Unnamed: 0                        405303
BOROUGH                           405303
NEIGHBORHOOD                      405303
BUILDING CLASS CATEGORY           405303
TAX CLASS AT PRESENT              405303
BLOCK                             405303
LOT                               405303
EASE-MENT                         405303
BUILDING CLASS AT PRESENT         405303
ADDRESS                           405303
APARTMENT NUMBER                  405303
ZIP CODE                          405303
RESIDENTIAL UNITS                 405303
COMMERCIAL UNITS                  405303
TOTAL UNITS                       405303
LAND SQUARE FEET                  405303
GROSS SQUARE FEET                 405303
YEAR BUILT                        405303
TAX CLASS AT TIME OF SALE         405303
BUILDING CLASS AT TIME OF SALE    405303
SALE PRICE                        405303
SALE DATE                         405303
dtype: int64